In [32]:
import sys, pickle, json, os, re
import numpy as np
from collections import defaultdict
from pprint import pprint
from typing import List, Dict, Any
sys.path.append('../src/')
sys.path.append("../../scripts/")
from utils import find_repo_root

import matplotlib.pyplot as plt
from plt_config import use_latex, TU_GREY, TU_RED
use_latex()
project_root_dir = find_repo_root()
print("Project root directory:", project_root_dir)

Project root directory: /home/etschgi1/REPOS/Masterarbeit


In [33]:
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py"
ray_folder = "/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py"
eval_name = "eval_res_reeval.json"
trials = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, eval_name)):
        with open(os.path.join(sub_folder_path, eval_name)) as f:
            eval_res = json.load(f)
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        trials.append((params, eval_res, sub_folder_path))

# alternative from param_paths_override: 
paths = ["993a0_00079_79",
"993a0_00144_144",
"993a0_00070_70",
"993a0_00009_9",
"993a0_00082_82",
"993a0_00149_149",
"993a0_00121_121",
"993a0_00158_158",
"993a0_00157_157",
"993a0_00014_14"]
trials = []
folders = [f for f in os.listdir(ray_folder) if any([re.search(p, f) for p in paths])]
for folder in folders: 
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, eval_name)):
        with open(os.path.join(sub_folder_path, eval_name)) as f:
            eval_res = json.load(f)
    else: 
        with open(os.path.join(sub_folder_path, "eval_res.json")) as f:
            eval_res = json.load(f)
    with open(os.path.join(sub_folder_path, "params.json")) as f:
        params = json.load(f)
    trials.append((params, eval_res, sub_folder_path))
len(trials)

10

In [34]:
# add 0 iterations if not present
for i, (params, eval_res, sub_folder_path) in enumerate(trials):
    if "iterations" not in eval_res["cummulative_stats"]:
        cum_stats_dict = eval_res["cummulative_stats"]
        cum_stats_dict["iterations"] = {}
        cum_stats_dict["iterations"]["mean"] = 0
        cum_stats_dict["iterations"]["std"] = 0
        eval_res["cummulative_stats"] = cum_stats_dict
        trials[i] = (params, eval_res, sub_folder_path)



In [35]:
trials[0][0].keys()

dict_keys(['batch_size', 'data_aug_factor', 'edge_threshold_val', 'grace_epochs', 'hidden_dim', 'loss_on_full_matrix', 'lr', 'lr_cooldown', 'lr_factor', 'lr_min', 'lr_patience', 'lr_threshold', 'message_net_dropout', 'message_net_layers', 'message_passing_steps', 'num_epochs', 'weight_decay'])

In [ ]:
trials.sort(key=lambda x:x[1]["cummulative_stats"]["iterations"]["mean"])
# trials.sort(key=lambda x:x[0]["edge_threshold_val"])#["iterations"]["mean"])
for trial in trials: 
    print(trial[0])
    print(trial[1]["cummulative_stats"])
    print("---")

{'batch_size': 32, 'data_aug_factor': 1.3282264678043476, 'edge_threshold_val': 2.698292377681005, 'grace_epochs': 5, 'hidden_dim': 512, 'loss_on_full_matrix': True, 'lr': 0.0006676585267638678, 'lr_cooldown': 2, 'lr_factor': 0.5, 'lr_min': 1e-06, 'lr_patience': 3, 'lr_threshold': 0.001, 'message_net_dropout': 0.1042200059278637, 'message_net_layers': 5, 'message_passing_steps': 2, 'num_epochs': 30, 'weight_decay': 3.7507630020426994e-05}
{'energy_abs': {'mean': 1.2322999515678248, 'std': 0.8441153664178067}, 'iterations': {'mean': 12.0, 'std': 0.8717797887081347}, 'energy_rel': {'mean': 0.0007020106719265429, 'std': 0.00047967971745986794}, 'diis': {'mean': 0.030598772220472806, 'std': 0.00393040069576333}, 'rmse': {'mean': 0.008352707021353305, 'std': 0.0005763196245690529}}
---
{'batch_size': 16, 'data_aug_factor': 2.3925915909864983, 'edge_threshold_val': 3.0204991769802234, 'grace_epochs': 5, 'hidden_dim': 512, 'loss_on_full_matrix': True, 'lr': 0.00034888978367533764, 'lr_cooldow

In [37]:
table_data = []
Hartree_conv = 1000 # mH
for i, (params, metrics, path) in enumerate(trials):
    metrics = metrics["cummulative_stats"]
    row = {
        'ID': i,
        'Iterations': f"{float(metrics['iterations']['mean']):.1f}({float(metrics['iterations']['std']):.1f})",
        r"$\Delta E_\text{HF}$": f"{float(metrics['energy_abs']['mean']):.3f}({float(metrics['energy_abs']['std']):.3f})",
        r"$\delta E_\text{HF}$": f"{float(metrics['energy_rel']['mean']):.4f}({float(metrics['energy_rel']['std']):.4f})",
        'DIIS error': f"{float(metrics['diis']['mean']):.3f}({float(metrics['diis']['std']):.4f})",
        r"$\overline{\text{RMSE}}$": f"{float(metrics['rmse']['mean']):.4f}({float(metrics['rmse']['std']):.4f})"
    }
    table_data.append(row)
    print(path)
import pandas as pd
df = pd.DataFrame(table_data)

latex = df.to_latex(index=False)
print(latex)

/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py/hyperopt_train_993a0_00149_149_batch_size=32,data_aug_factor=1.3282,edge_threshold_val=2.6983,hidden_dim=512,lr=0.0007,message_net_2025-07-12_05-52-52
/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py/hyperopt_train_993a0_00014_14_batch_size=16,data_aug_factor=2.3926,edge_threshold_val=3.0205,hidden_dim=512,lr=0.0003,message_net__2025-07-11_13-21-40
/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py/hyperopt_train_993a0_00079_79_batch_size=32,data_aug_factor=1.9081,edge_threshold_val=2.1113,hidden_dim=256,lr=0.0003,message_net__2025-07-11_22-04-17
/home/etschgi1/REPOS/Masterarbeit/3_studies/Block_guessing/6-31g_testing/tune_logs/MGNN_hyp_small_full_mat_loss.py/hyperopt_train_993a0_00070_70_batch_size=32,data_aug_factor=2.3284,edge_threshold_val=2

In [38]:
table_data = []
for i, (params, metrics, path) in enumerate(trials):
    metrics = metrics["cummulative_stats"]
    row = {
        'Parameter': int(i),
        'Hidden Dimension': int(params["hidden_dim"]),
        'Batch Size': int(params["batch_size"]),
        "Data aug.": f'{float(params["data_aug_factor"]):.2f}',
        "Edge thres.": f'{float(params["edge_threshold_val"]):.2f}',
        "Message p. steps": int(params["message_passing_steps"]),
        "Message Net Dropout": f'{float(params["message_net_dropout"]):.2f}',
        "Message Net Layers": int(params["message_net_layers"]),
        "Learn Rate": f'{float(params["lr"]):.2e}',
        "Weight Decay": f'{float(params["weight_decay"]):.2e}',
    }
    table_data.append(row)

import pandas as pd
df = pd.DataFrame(table_data).T


latex = df.to_latex(index=True)
print(latex)

\begin{tabular}{lllllllllll}
\toprule
 & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\midrule
Parameter & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
Hidden Dimension & 512 & 512 & 256 & 256 & 256 & 256 & 256 & 128 & 128 & 512 \\
Batch Size & 32 & 16 & 32 & 32 & 8 & 32 & 8 & 32 & 32 & 8 \\
Data aug. & 1.33 & 2.39 & 1.91 & 2.33 & 1.50 & 1.67 & 1.24 & 2.45 & 1.50 & 1.71 \\
Edge thres. & 2.70 & 3.02 & 2.11 & 2.05 & 3.81 & 2.14 & 2.17 & 2.12 & 2.11 & 3.81 \\
Message p. steps & 2 & 2 & 3 & 3 & 2 & 3 & 2 & 2 & 4 & 4 \\
Message Net Dropout & 0.10 & 0.25 & 0.22 & 0.11 & 0.19 & 0.18 & 0.26 & 0.23 & 0.29 & 0.22 \\
Message Net Layers & 5 & 5 & 5 & 3 & 3 & 3 & 5 & 3 & 5 & 3 \\
Learn Rate & 6.68e-04 & 3.49e-04 & 2.61e-04 & 1.16e-04 & 1.43e-03 & 3.54e-04 & 5.15e-03 & 4.36e-03 & 3.97e-03 & 8.77e-03 \\
Weight Decay & 3.75e-05 & 6.32e-05 & 2.97e-04 & 2.35e-05 & 2.10e-04 & 5.21e-05 & 2.49e-04 & 4.43e-05 & 9.97e-04 & 8.30e-04 \\
\bottomrule
\end{tabular}



In [45]:
data = []
for folder in os.listdir(ray_folder):
    sub_folder_path = os.path.join(ray_folder, folder)
    if os.path.exists(os.path.join(sub_folder_path, "result.json")):
        with open(os.path.join(sub_folder_path, "result.json")) as f:
            result = f.readlines()
        result = [json.loads(line) for line in result if line.strip()]
        lastline = result[-1]
        with open(os.path.join(sub_folder_path, "params.json")) as f:
            params = json.load(f)
        data.append((folder, params, lastline["loss"]))

In [44]:
sorted_data = sorted(data, key=lambda x: x[1])
for path, loss in sorted_data[:10]:
    print(f"{path}: {loss:.4f}")

hyperopt_train_993a0_00079_79_batch_size=32,data_aug_factor=1.9081,edge_threshold_val=2.1113,hidden_dim=256,lr=0.0003,message_net__2025-07-11_22-04-17: 1262.2497
hyperopt_train_993a0_00144_144_batch_size=32,data_aug_factor=1.4980,edge_threshold_val=2.1125,hidden_dim=128,lr=0.0040,message_net_2025-07-12_05-17-37: 1288.5993
hyperopt_train_993a0_00070_70_batch_size=32,data_aug_factor=2.3284,edge_threshold_val=2.0468,hidden_dim=256,lr=0.0001,message_net__2025-07-11_20-30-26: 1294.2693
hyperopt_train_993a0_00009_9_batch_size=32,data_aug_factor=2.4478,edge_threshold_val=2.1235,hidden_dim=128,lr=0.0044,message_net_d_2025-07-11_12-44-42: 1298.3422
hyperopt_train_993a0_00082_82_batch_size=32,data_aug_factor=1.6722,edge_threshold_val=2.1384,hidden_dim=256,lr=0.0004,message_net__2025-07-11_22-25-17: 1302.5625
hyperopt_train_993a0_00150_150_batch_size=32,data_aug_factor=1.5349,edge_threshold_val=2.1448,hidden_dim=128,lr=0.0013,message_net_2025-07-12_06-01-36: 1320.4846
hyperopt_train_993a0_00095_9

In [59]:
# data aug in range
data_n = [x for x in data if 2.32 <= x[1]["data_aug_factor"] <= 2.34]
data_n

[('hyperopt_train_993a0_00070_70_batch_size=32,data_aug_factor=2.3284,edge_threshold_val=2.0468,hidden_dim=256,lr=0.0001,message_net__2025-07-11_20-30-26',
  {'batch_size': 32,
   'data_aug_factor': 2.32843109208835,
   'edge_threshold_val': 2.046848789531119,
   'grace_epochs': 5,
   'hidden_dim': 256,
   'loss_on_full_matrix': True,
   'lr': 0.0001160861493167114,
   'lr_cooldown': 2,
   'lr_factor': 0.5,
   'lr_min': 1e-06,
   'lr_patience': 3,
   'lr_threshold': 0.001,
   'message_net_dropout': 0.10802434241852031,
   'message_net_layers': 3,
   'message_passing_steps': 3,
   'num_epochs': 30,
   'weight_decay': 2.3506875491839735e-05},
  1294.2693481445312),
 ('hyperopt_train_993a0_00136_136_batch_size=32,data_aug_factor=2.3237,edge_threshold_val=2.8383,hidden_dim=256,lr=0.0033,message_net_2025-07-12_04-26-10',
  {'batch_size': 32,
   'data_aug_factor': 2.323700581278806,
   'edge_threshold_val': 2.838342279688234,
   'grace_epochs': 5,
   'hidden_dim': 256,
   'loss_on_full_matri